Start by creating the schema in Postgres if it does not exist.

In [ ]:
CREATE schema if not exists sharks 

Sanity check the columns we want to create in the table schema for the two tables

In [ ]:
select * from sharks_locs.csv

In [ ]:
select * from sharks_metadata.csv

We'll use the metadata as the primary table, then have the locations table foreign key to the metadata table through the id.

In [ ]:
CREATE table if not exists sharks.metadata (
    id int primary key,
    name text unique not null,
    gender text,
    weight int,
    length int,
    species text not null)

In [ ]:
%create_or_replace_data_view sharks_metadata.csv sharks.import_table -c "Ocearch Postgres"



In [ ]:
INSERT into sharks.metadata SELECT * from "sharks.import_table"

In [ ]:
SELECT * FROM sharks.metadata

Now we can create the locations table foreign key'd to metadata table

In [ ]:
locs_df.head()

In [ ]:
CREATE table if not exists sharks.locs (
    id serial primary key,
    shark_id int not null references sharks.metadata(id),
    ts timestamptz not null,
    latitude float not null,
    longitude float not null
)

In [ ]:
%create_or_replace_data_view sharks_locs.csv sharks.import_table -c "Ocearch Postgres"

In [ ]:
SELECT * FROM "sharks.import_table" LIMIT 10

In [ ]:
# double check date formatting
!head -5 sharks_locs.csv 

In [ ]:
INSERT into sharks.locs (shark_id, ts, latitude, longitude)
    SELECT id, TO_TIMESTAMP(datetime, 'YYYY-MM-DD HH24:mi:ss'), latitude, longitude from "sharks.import_table"

In [ ]:
SELECT m.name, count(*)
FROM sharks.metadata m
JOIN sharks.locs l
ON (m.id = l.shark_id)